Intuition for "Factor 2" bug:
- We compare estimated propensity score to the true propensity score
- In large samples it is symetrically centered around the true estimate
- Thus the probability that the spline receives a weight is 0.5
- Hence, on average we only give half the required weight to the required observations
- This does not affect larger splines (for higher u intervals) because they require propensity scores that are a lot larger (or smaller) than the current one being estimated
- Note, this should *not* happen if we use the estimated propensity score only
- Now, using targets very close shouldn't change anything really?
  - We are just using more splines, should this have a large effect?

In [21]:
# Check the consistency of individual estimators (in particular gamma weights)
%reload_ext autoreload
%autoreload 2

from iv_part_id_estimation import *
from iv_part_id import *
from plot_funcs import *
from funcs import *
from simulate_data import *
from sim_funcs import *
from table_by_tol import *
import os
import time
import os
import itertools

# Set seed
np.random.seed(12598612)

In [3]:
# Set the parameters of the model
identif = ["cross","cross","cross","cross","cross","cross"] # the set of identified estimands
basis = "cs" # the basis function to use for approximation
shape = None # shape restriction
target = "late" # the target estimand
u_lo_target = 0.35 # if target = LATE: lower u
u_hi_target = 0.9 # if target = LATE: upper u
supp_z = [np.array([0, 1, 2])] # if identif = iv_slope: support of the instrument
prop_z = [np.array([0.35, 0.6, 0.7])] # if identif = iv_slope: propensity given the instrument
f_z = [np.array([0.5, 0.4, 0.1])] # if identif = iv_slope: probability mass function of the instrument
dz_cross = dz_cross = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)] # if identif = cross: the cross moment
m0_dgp = m0_dgp
m1_dgp = m1_dgp

u_part = [np.array([0, 0.35, 0.6, 0.7, 0.9, 1])] # if basis = cs: partition of u in [0,1]
u_part


[array([0.  , 0.35, 0.6 , 0.7 , 0.9 , 1.  ])]

In [4]:
true_gamma = compute_gamma_df(
            "cross",
            basis="cs",
            u_part=u_part,
            supp_z=supp_z[0],
            prop_z=prop_z[0],
            f_z=f_z[0],
            dz_cross=dz_cross[0], analyt_int = True
            )
true_gamma

,gamma
THETA,


In [5]:
data = simulate_data(1000, supp_z[0], f_z[0], prop_z[0])
y = data[:, 0]
d = data[:, 1]
z = data[:, 2]
np.unique(d), np.unique(z)


u_part_est = estimate_prop_z(z,d)
u_part_est = np.append(u_part_est, [np.min(u_part_est), u_hi_target]) # add target cutoffs to partition
u_part_est = np.unique(np.sort(u_part_est)) # make sure partition is ordered and remove duplicates
u_part_est = np.insert(u_part_est, 0, 0) # add 0 to partition at beginning
u_part_est = np.append(u_part_est, 1) # add 1 to partition
u_part_est = [u_part_est]
u_part_est


g_estim = compute_gamma_df_estimation(y, z, d, "cross", "cs", dz_cross=[0,1], u_part = u_part[0]) 
g_true = compute_gamma_df("cross", "cs", dz_cross=[0,1], u_part = u_part[0], supp_z = supp_z[0], prop_z = prop_z[0], f_z = f_z[0], analyt_int = True)
g_estim, g_true

(           gamma
 THETA           
 theta0_0  0.0000
 theta0_1  0.0000
 theta0_2  0.0000
 theta0_3  0.0812
 theta0_4  0.0406
 theta1_0  0.0000
 theta1_1  0.0000
 theta1_2  0.0000
 theta1_3  0.0000
 theta1_4  0.0000,
           gamma
 THETA          
 theta0_0   0.00
 theta0_1   0.00
 theta0_2   0.04
 theta0_3   0.08
 theta0_4   0.04
 theta1_0   0.00
 theta1_1   0.00
 theta1_2   0.00
 theta1_3   0.00
 theta1_4   0.00)

In [6]:
def sim_gamma_df_estimation(N, R, dz_cross, supp_z, f_z, prop_z, u_part):
    
    n_theta = (len(u_part) - 1)*2

    out = np.zeros((R, n_theta))    

    for r in range(R):
        data = simulate_data(N, supp_z, f_z, prop_z)
        
        y = data[:, 0]
        d = data[:, 1]
        z = data[:, 2]

        df = compute_gamma_df_estimation(y, z, d, "cross", "cs", dz_cross=dz_cross, u_part = u_part)     

        for i in range(n_theta):   
            if i <= (n_theta // 2) - 1: 
                out[r, i] = df.loc["theta0_" + str(i)]
            else:
                out[r, i] = df.loc["theta1_" + str(i - (n_theta // 2))]

    gamma_estim = pd.DataFrame(out, 
        columns = [
            "theta0_0", 
            "theta0_1", 
            "theta0_2", 
            "theta0_3", 
            "theta0_4", 
            "theta1_0", 
            "theta1_1", 
            "theta1_2", 
            "theta1_3",
            "theta1_4", 
            ]
            )

    # Get the means of each column
    return gamma_estim.mean(axis = 0)

In [7]:
sim_gamma_df_estimation(10000, 10000, [0,1], supp_z[0], f_z[0], prop_z[0], u_part[0])

KeyboardInterrupt: 

In [13]:
# Repeat sim_gamma_df_estimation for all dz_cross
dz_cross = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

estimates = [sim_gamma_df_estimation(10000, 1000, dz, supp_z[0], f_z[0], prop_z[0], u_part = u_part[0]) 
    for dz in dz_cross]

In [19]:
# Combine all df in estimates into one big df with columns corresponding to dz_cross
estimates = pd.concat(estimates, axis = 1)
estimates.columns = ["dz_cross_" + str(dz) for dz in dz_cross]
estimates

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [21]:
# Repeat the same for true gamma
truth = [compute_gamma_df(
            "cross",
            basis="cs",
            u_part=u_part[0],
            supp_z=supp_z[0],
            prop_z=prop_z[0],
            f_z=f_z[0],
            dz_cross=dz, analyt_int = True
            ) for dz in dz_cross]

In [22]:
# Combine all df in truth into one big df with columns corresponding to dz_cross
truth = pd.concat(truth, axis = 1)
truth.columns = ["dz_cross_" + str(dz) for dz in dz_cross]
truth

,"dz_cross_(0, 0)","dz_cross_(0, 1)","dz_cross_(0, 2)","dz_cross_(1, 0)","dz_cross_(1, 1)","dz_cross_(1, 2)"
THETA,,,,,,
theta0_0,0.000,0.00,0.00,0.000,0.00,0.000
theta0_1,0.125,0.00,0.00,0.000,0.00,0.000
theta0_2,0.050,0.04,0.00,0.000,0.00,0.000
theta0_3,0.100,0.08,0.02,0.000,0.00,0.000
theta0_4,0.050,0.04,0.01,0.000,0.00,0.000
theta1_0,0.000,0.00,0.00,0.175,0.14,0.035
theta1_1,0.000,0.00,0.00,0.000,0.10,0.025
theta1_2,0.000,0.00,0.00,0.000,0.00,0.010
theta1_3,0.000,0.00,0.00,0.000,0.00,0.000


In [23]:
estimates

,"dz_cross_(0, 0)","dz_cross_(0, 1)","dz_cross_(0, 2)","dz_cross_(1, 0)","dz_cross_(1, 1)","dz_cross_(1, 2)"
theta0_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
theta0_1,0.063990,0.000000,0.000000,0.000000,0.000000,0.000000
theta0_2,0.050007,0.019134,0.000000,0.000000,0.000000,0.000000
theta0_3,0.100014,0.080035,0.010198,0.000000,0.000000,0.000000
theta0_4,0.050007,0.040017,0.010009,0.000000,0.000000,0.000000
theta1_0,0.000000,0.000000,0.000000,0.086255,0.140013,0.034973
theta1_1,0.000000,0.000000,0.000000,0.000000,0.050571,0.024981
theta1_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.004850
theta1_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
theta1_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
estimates / truth

,"dz_cross_(0, 0)","dz_cross_(0, 1)","dz_cross_(0, 2)","dz_cross_(1, 0)","dz_cross_(1, 1)","dz_cross_(1, 2)"
theta0_0,NaN,NaN,NaN,NaN,NaN,NaN
theta0_1,0.511921,NaN,NaN,NaN,NaN,NaN
theta0_2,1.000140,0.478338,NaN,NaN,NaN,NaN
theta0_3,1.000140,1.000432,0.509918,NaN,NaN,NaN
theta0_4,1.000140,1.000432,1.000876,NaN,NaN,NaN
theta1_0,NaN,NaN,NaN,0.492884,1.000090,0.999222
theta1_1,NaN,NaN,NaN,NaN,0.505715,0.999222
theta1_2,NaN,NaN,NaN,NaN,NaN,0.484962
theta1_3,NaN,NaN,NaN,NaN,NaN,NaN
theta1_4,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
identif = ["cross","cross","cross","cross","cross","cross"] # the set of identified estimands
basis = "cs" # the basis function to use for approximation
supp_z = [np.array([0, 1, 2])] # if identif = iv_slope: support of the instrument
prop_z = [np.array([0.35, 0.6, 0.7])] # if identif = iv_slope: propensity given the instrument
f_z = [np.array([0.5, 0.4, 0.1])] # if identif = iv_slope: probability mass function of the instrument
dz_cross = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)] # if identif = cross: the cross moment


iv_part_id_estimation(
    y=y,
    z=z,
    d=d,
    target = "late", 
    identif = identif,
    basis="cs", 
    u_lo_target = 0.35, 
    u_hi_target = 0.9,
    dz_cross = dz_cross,
    tol = 1/1000,
    )

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 0.0002206601603
5 simplex iterations
0 barrier iterations
Objective is: 0.000220660160288615
argmax:           Theta_val.val
theta0_0       0.000000
theta0_1       0.000000
theta0_2       0.254689
theta0_3       1.000000
theta0_4       0.141222
theta0_5       1.000000
theta1_0       0.672545
theta1_1       0.000000
theta1_2       0.626846
theta1_3       0.190645
theta1_4       0.000000
theta1_5       0.000000
reset;
set THETA;
set IDENTIF_0;
set IDENTIF_1;
set IDENTIF_2;
set IDENTIF_3;
set IDENTIF_4;
set IDENTIF_5;
param gamma {THETA};
param gamma_ident_0 {THETA};
param val_identif_0 {IDENTIF_0};
param gamma_ident_1 {THETA};
param val_identif_1 {IDENTIF_1};
param gamma_ident_2 {THETA};
param val_identif_2 {IDENTIF_2};
param gamma_ident_3 {THETA};
param val_identif_3 {IDENTIF_3};
param gamma_ident_4 {THETA};
param val_identif_4 {IDENTIF_4};
param gamma_ident_5 {THETA};
param val_identif_5 {IDENTIF_5};
var Theta_val {j in THETA} >= 0,

((0.000220660160288615,
            Theta_val.val
  theta0_0       0.000000
  theta0_1       0.000000
  theta0_2       0.254689
  theta0_3       1.000000
  theta0_4       0.141222
  theta0_5       1.000000
  theta1_0       0.672545
  theta1_1       0.000000
  theta1_2       0.626846
  theta1_3       0.190645
  theta1_4       0.000000
  theta1_5       0.000000),
 (0.44612808970347434,
            Theta_val.val
  theta0_0       0.000000
  theta0_1       0.000000
  theta0_2       0.487505
  theta0_3       0.000000
  theta0_4       0.141222
  theta0_5       1.000000
  theta1_0       0.672545
  theta1_1       0.000000
  theta1_2       0.626846
  theta1_3       0.380099
  theta1_4       1.000000
  theta1_5       0.000000),
 (-0.06671853168335698,
            Theta_val.val
  theta0_0       0.000000
  theta0_1       0.000000
  theta0_2       0.254689
  theta0_3       1.000000
  theta0_4       0.141222
  theta0_5       1.000000
  theta1_0       0.672545
  theta1_1       1.000000
  theta1_2     

In [22]:
# N = 1000
N = 1000
R = 10

def get_tolerances(N):
    return [N**(-1)]

# Target parameter
target = "late"
u_lo_target = 0.2
u_hi_target = 0.9

basis = "cs" # the basis function to use for approximation
supp_z = np.array([0, 1, 2]) # if identif = iv_slope: support of the instrument
prop_z = np.array([0.35, 0.6, 0.7]) # if identif = iv_slope: propensity given the instrument
f_z = np.array([0.5, 0.4, 0.1]) # if identif = iv_slope: probability mass function of the instrument
dz_cross = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)] # if identif = cross: the cross moment

identif = ["cross","cross","cross","cross","cross","cross"] # the set of identified estimands

bld = "C:/Users/budde/OneDrive/phd_bgse/courses/topics_metrics/topics_sim/bld/"
# Add subfolder to bld with systemtime and create if it does not exist
bld = bld + "sim_parallel_" + time.strftime("%Y%m%d-%H%M%S")

simulation(N, R, supp_z, f_z, prop_z, target, identif, basis, 0.0001, u_lo_target,
                                                 u_hi_target, dz_cross,
                                                  False, True, bld)





Run: 1






ValueError: a must be greater than 0 unless no samples are taken

In [24]:
import numpy as np

num_points = 11  # The number of points to generate
start = 0  # The starting point
stop = 1  # The stopping point

# Generate the list of equally spaced numbers
list_of_numbers = np.linspace(start, stop, num_points)

print(list_of_numbers)  # Output: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556 0.66666667 0.77777778 0.88888889 1.        ]

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
